# Nano-bigram forecasting

Using series data to predict an item 5 time steps ahead.

In [1]:
import numpy as np
import torch
from tqdm import tqdm
import torch.nn as nn
from torch.nn import functional as F
import os.path

In [2]:
# Variables

from config import KEY, PERIOD, TARGET_PERIOD, NORM_DATA, STORE_PATH

In [3]:

# Build fp
fields = [KEY, str(PERIOD), str(TARGET_PERIOD)]
if NORM_DATA:
    fields.append("norm")
DATA_FP = "_".join(fields)+".np"
VAL_DATA_FP = "_".join(fields+["val"])+".np"
MODEL_FP = "_".join(fields)+".pt"
fp = os.path.join(STORE_PATH, DATA_FP)
val_fp = os.path.join(STORE_PATH, VAL_DATA_FP)
model_fp = os.path.join(STORE_PATH, MODEL_FP)

In [4]:
# Import the data
data = np.load(fp)
val_data = np.load(val_fp)

In [5]:
data = np.where(np.isnan(data), 0, data)
val_data = np.where(np.isnan(val_data), 0, val_data)

In [6]:

# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = PERIOD # what is the maximum context length for predictions?
max_epochs = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 10
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
vocab_size = 1
# ------------

torch.manual_seed(1705)

In [7]:
# Train and test splits
train_data = torch.tensor(data, dtype=torch.float32)
val_data = torch.tensor(val_data, dtype=torch.float32)
print("train and val data shape:", train_data.shape, val_data.shape)

In [8]:
# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    idx = torch.randperm(data.shape[0])
    data = data[idx]
    x = data[:, :block_size]
    y = data[:, 1:block_size+1]
    ix = torch.randint(len(data), (batch_size,))
    x = x[ix,:]
    y = y[ix,:]
    # ix = torch.randint(len(data) - block_size, (batch_size,))
    # x = torch.stack([data[i:i+block_size] for i in ix])
    # y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [9]:

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

In [10]:
class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

In [11]:
class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        return self.net(x)

In [12]:

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

In [13]:
class ForecastTransformer(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Linear(block_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape
        # B == Batch
        # T == Input tensor length
        # C == Channels (positional embedding size)
        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx).unsqueeze(1) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device, dtype=torch.int64)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T)
            targets = targets.view(B*T)
            loss = F.mse_loss(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, _ = self(idx_cond)
            # append sampled index to the running sequence
            idx = torch.cat((idx, logits[:, :, -1][:, -1].unsqueeze(1)), dim=1) # (B, T+1)
        return idx

In [14]:
class EarlyStopper:
    def __init__(self, patience=1, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.min_validation_loss = np.inf

    def early_stop(self, validation_loss):
        if np.isnan(validation_loss):
            return True
        if validation_loss < self.min_validation_loss:
            self.min_validation_loss = validation_loss
            self.counter = 0
        elif validation_loss > (self.min_validation_loss + self.min_delta):
            self.counter += 1
            if self.counter >= self.patience:
                return True
        return False

In [15]:

model = ForecastTransformer()
m = model.to(device)
# print the number of parameters in the model
print(f"{sum(p.numel() for p in m.parameters()):,} parameters")

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

# Loss estimation
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

early_stopper = EarlyStopper(patience=3, min_delta=10)
for epoch in tqdm(range(max_epochs), total=max_epochs, desc="Epoch"):

    # every once in a while evaluate the loss on train and val sets
    if epoch % eval_interval == 0 or epoch == max_epochs - 1:
        losses = estimate_loss()
        print(f"step {epoch}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
        if early_stopper.early_stop(losses['val']):             
            print("Early stopping")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()


210,945 parameters


Epoch:   0%|          | 0/5000 [00:00<?, ?it/s]

step 0: train loss 0.4705, val loss 0.4827


Epoch:   2%|▏         | 101/5000 [00:27<1:10:04,  1.17it/s]

step 100: train loss 0.0551, val loss 0.0505


...

Epoch:  98%|█████████▊| 4901/5000 [19:27<01:21,  1.21it/s]

step 4900: train loss 0.0025, val loss 0.0028


Epoch: 100%|█████████▉| 4999/5000 [19:48<00:00,  4.22it/s]

step 4999: train loss 0.0026, val loss 0.0023


Epoch: 100%|██████████| 5000/5000 [19:51<00:00,  4.20it/s]


In [16]:
# Save the model
torch.save(model, model_fp)